In [ ]:
! pip install -U git+https://github.com/optuna/optuna.git -q

In [ ]:
!wget https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/YmyigRey-Nic9Q

In [ ]:
!mv YmyigRey-Nic9Q dataset.zip

In [ ]:
!unzip dataset.zip

In [ ]:
!pip install kaggle

In [ ]:
!cp /kaggle/input/json-api/kaggle.json /root/.kaggle

In [ ]:
!kaggle datasets download -d yurybelousov/preprocessed-citymobil

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/final-etarta/validation.csv
/kaggle/input/final-etarta/test_additional.csv
/kaggle/input/final-etarta/test.csv
/kaggle/input/final-etarta/train.csv/train.csv
/kaggle/input/json-api/kaggle.json
/kaggle/input/training/hpo_Booster.joblib
/kaggle/input/training/simple_LGBMRegressor.txt
/kaggle/input/training/hpo_Booster.txt
/kaggle/input/training/__notebook__.ipynb
/kaggle/input/training/study.joblib
/kaggle/input/training/custom.css
/kaggle/input/training/__results__.html
/kaggle/input/training/__output__.json
/kaggle/input/training/simple_LGBMRegressor.joblib
/kaggle/input/training/boostings/3.pkl
/kaggle/input/training/boostings/14.pkl
/kaggle/input/training/boostings/5.pkl
/kaggle/input/training/boostings/21.pkl
/kaggle/input/training/boostings/56.pkl
/kaggle/input/training/boostings/59.pkl
/kaggle/input/training/boostings/13.pkl
/kaggle/input/training/boostings/62.pkl
/kaggle/input/training/boostings/55.pkl
/kaggle/input/training/boostings/37.pkl
/kaggle/input/training/b

In [2]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

def mean_absolute_percentage_error(y_true, y_pred):
    mask = (y_true != 0)
    return (np.fabs(y_true - y_pred) / y_true)[mask].mean() * 100

def calculate_metrics(y_true, y_pred):
    print(f"MAPE: {mean_absolute_percentage_error(y_true, y_pred)}")
    print(f"mse: {mean_squared_error(y_true, y_pred)}")
    print(f"r2: {r2_score(y_true, y_pred)}")

In [3]:
import gc
import json
from datetime import timedelta
from pathlib import Path

import category_encoders as ce

from joblib import dump
from tqdm.autonotebook import tqdm
tqdm.pandas()

from sklearn.cluster import MiniBatchKMeans
import catboost as cb



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


In [4]:
data_path = Path('/') / 'kaggle' / 'input' / 'yury-data' / 'preprocessed_citymobil'

In [5]:
train = pd.read_csv(data_path / 'train.csv', index_col='Id')
valid = pd.read_csv(data_path / 'valid.csv', index_col='Id')
test = pd.read_csv(data_path / 'test.csv', index_col='Id')

In [6]:
train.head()

,main_id_locality,ETA,RTA,OrderedDate,latitude,del_latitude,longitude,del_longitude,EDA,RDA,...,work_avg_speed_at_14,work_avg_speed_at_15,work_avg_speed_at_16,work_avg_speed_at_17,work_avg_speed_at_18,work_avg_speed_at_19,work_avg_speed_at_20,work_avg_speed_at_21,work_avg_speed_at_22,work_avg_speed_at_23
Id,,,,,,,,,,,,,,,,,,,,,
0,1078,226.0,188.0,2020-02-12 19:12:06,55.826019,55.825581,49.134529,49.126949,1.0,1.0,...,0.007848,0.007812,0.007144,0.00629,0.006826,0.008095,0.008686,0.009017,0.00931,0.009489
1,1078,718.0,725.0,2020-02-12 19:12:22,55.795502,55.820911,49.131470,49.115360,5.0,6.0,...,0.007848,0.007812,0.007144,0.00629,0.006826,0.008095,0.008686,0.009017,0.00931,0.009489
2,1078,612.0,764.0,2020-02-12 19:12:44,55.791050,55.819962,49.226070,49.176628,5.0,5.0,...,0.007848,0.007812,0.007144,0.00629,0.006826,0.008095,0.008686,0.009017,0.00931,0.009489
3,1078,1560.0,1412.0,2020-02-12 19:12:44,55.753899,55.824680,49.188519,49.093700,13.0,14.0,...,0.007848,0.007812,0.007144,0.00629,0.006826,0.008095,0.008686,0.009017,0.00931,0.009489
4,1078,1528.0,893.0,2020-02-12 19:12:45,55.822361,55.786758,49.069092,49.143501,9.0,10.0,...,0.007848,0.007812,0.007144,0.00629,0.006826,0.008095,0.008686,0.009017,0.00931,0.009489


In [7]:
def get_city_speed(x):
    if x['is_night']:
        return x[f"night_avg_speed_at_{x['hour']}"]
    
    if x['is_rush_hour']:
        return x[f"rush_avg_speed_at_{x['hour']}"]
    
    if x['is_weekend']:
        return x[f"weekend_avg_speed_at_{x['hour']}"]
    
    return x[f"work_avg_speed_at_{x['hour']}"]

In [8]:
def get_estimated_speed(x):
    return x['EDA'] / x['ETA']

In [9]:
def speed_preprocessing(df):
    df['current_city_speed'] = df.progress_apply(get_city_speed, axis=1)
    df['estimated_speed'] = df.apply(get_estimated_speed, axis=1)
    df['speed_difference'] = df['current_city_speed'] - df['estimated_speed']
    df['speed_ratio'] = df['current_city_speed'] / df['estimated_speed']
    df['detour_time'] = df['detour'] * df['estimated_speed']
    return df

In [10]:
train = speed_preprocessing(train)
valid = speed_preprocessing(valid)
test = speed_preprocessing(test)

In [11]:
df = train

In [12]:
df_test = test
df_val = valid

In [ ]:
df['diff'] = np.abs(df['ETA'] - df['RTA'])

In [ ]:
m = np.mean(df['diff'])
s = np.std(df['diff'])
df = df[df['diff'] <= m + 3*s]
df.shape

In [13]:
coords = np.vstack((df[['latitude', 'longitude']].values,
                    df[['del_latitude', 'del_longitude']].values))

In [14]:
sample_ind = np.random.permutation(len(coords))[:500000]
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000).fit(coords[sample_ind])

In [15]:
df.loc[:, 'pickup_cluster'] = kmeans.predict(df[['latitude', 'longitude']])
df.loc[:, 'dropoff_cluster'] = kmeans.predict(df[['del_latitude', 'del_longitude']])
df_test.loc[:, 'pickup_cluster'] = kmeans.predict(df_test[['latitude', 'longitude']])
df_test.loc[:, 'dropoff_cluster'] = kmeans.predict(df_test[['del_latitude', 'del_longitude']])

In [16]:
df_val.loc[:, 'pickup_cluster'] = kmeans.predict(df_val[['latitude', 'longitude']])
df_val.loc[:, 'dropoff_cluster'] = kmeans.predict(df_val[['del_latitude', 'del_longitude']])

In [ ]:
set(train.columns) - set(test.columns)

Shuffle train

In [17]:
train = df

In [18]:
test = df_test
valid = df_val

In [ ]:
train = train.sample(frac=1)

In [19]:
columns_for_training = test.columns.difference(['OrderedDate'])

In [20]:
X_train, y_train = train[columns_for_training], train['ETA'] - train['RTA']
X_valid, y_valid = valid[columns_for_training], valid['ETA'] - valid['RTA']

In [ ]:
feature_names = list(X_train.columns)

In [ ]:
feature_names

In [21]:
categorical_columns = ['main_id_locality', 'pickup_cluster', 'dropoff_cluster']

In [22]:
ordinal_columns = ['day_of_week', 'hour'] #month
encoder = ce.OrdinalEncoder(cols=ordinal_columns)

In [23]:
X_train = encoder.fit_transform(X_train)
X_valid = encoder.transform(X_valid)

In [26]:
model = cb.CatBoostRegressor(iterations=500, rsm=0.8, depth=12, learning_rate=0.037, early_stopping_rounds=100)
model.fit(X_train, y_train, categorical_columns + ordinal_columns,eval_set=(X_valid, y_valid), verbose=False, plot=True)

NameError: name 'X_val' is not defined

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
pd.options.display.max_rows = None

In [ ]:
X_train.head().T

In [ ]:
clf = LGBMRegressor(
            n_estimators=4000,
            learning_rate=0.03,
            num_leaves=30,
            colsample_bytree=.8,
            subsample=.9,
            max_depth=7,
            reg_alpha=.3,
            reg_lambda=.1,
            min_split_gain=.01,
            min_child_weight=2,
            verbose=2,  
)

In [ ]:
clf.fit(
        X_train, y_train, 
        eval_set= [(X_train, y_train), (X_valid, y_valid)], 
        eval_metric='rmse', verbose=100, early_stopping_rounds=500, 
        feature_name=feature_names, categorical_feature=categorical_columns
)

In [ ]:
def calculate_valid_metrics(clf, prefix=''):
    clf_name = f"{prefix}_{clf.__class__.__name__}"
    print(f'Evaluating {clf_name}')
    if hasattr(clf, 'booster_'):
        clf.booster_.save_model(f'{clf_name}.txt')
    elif hasattr(clf, 'save_model'):
        clf.save_model(f'{clf_name}.txt')
    dump(clf, f"{clf_name}.joblib")
    
    y_valid_pred = clf.predict(X_valid)
    print(f"Prediction diff: {y_valid_pred.mean()},\tTrue diff: {(valid['ETA'] - valid['RTA']).mean()}", end='\t')
    prediction_diff_bias = (valid['ETA'] - valid['RTA']).mean() - y_valid_pred.mean()
    print(f'Difference: {prediction_diff_bias}')
    
    valid['pred_diff'] = y_valid_pred
    valid['pred'] = valid['ETA'] - y_valid_pred
    
    prediction_diff_by_group = valid.groupby(by='main_id_locality').apply(lambda x: (x['ETA'] - x['RTA']).mean() - x['pred_diff'].mean()).to_frame(name = 'diff').reset_index()
    print(prediction_diff_by_group)
    
    print("Diff metrics")
    calculate_metrics(y_valid, y_valid_pred)
    
    print("True RTA metrics")
    calculate_metrics(valid['RTA'], valid['pred'])
    
    print("True RTA metrics with bias substracting")
    calculate_metrics(valid['RTA'], valid['pred'] - prediction_diff_bias)
    
    print("True RTA metrics with bias substracting by group")
    calculate_metrics(valid['RTA'], valid['pred'] - valid.merge(prediction_diff_by_group)['diff'])
    
    return prediction_diff_bias, prediction_diff_by_group

In [ ]:
prediction_diff_bias, prediction_diff_by_group = calculate_valid_metrics(clf, prefix='simple')

In [ ]:
df['RTA'].shape

In [ ]:
X_train['ETA'].shape

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(y_valid, clf.predict(X_valid))

In [ ]:
def prepate_test_submsission(clf, prediction_diff_bias, prediction_diff_by_group, prefix):
    prefix = Path(prefix)
    prefix.mkdir(exist_ok=True)
    
    X_test = test[columns_for_training]
    X_test = encoder.transform(X_test)
    
    test_pred = clf.predict(X_test)
    print(f"Mean of diff on test is {test_pred.mean()}")
    
    submission = test[['Id']].copy()
    
    submission['Prediction'] = test['ETA'] - test_pred
    submission.to_csv(prefix / "submission_pure.csv", index=False)
    
    submission['Prediction'] = test['ETA'] - test_pred - prediction_diff_bias
    submission.to_csv(prefix / "submission_without_bias.csv", index=False)
    
    submission['Prediction'] += valid['pred_diff'].mean() - test_pred.mean()
    submission.to_csv(prefix / "submission_without_bias_corrected.csv", index=False)
    
    submission['main_id_locality'] = test['main_id_locality']
    submission['Prediction'] = submission['Prediction'] - test.merge(prediction_diff_by_group)['diff']
    submission.drop(columns='main_id_locality', inplace=True)
    submission.to_csv(prefix / "submission_without_bias_grouped.csv", index=False)
    
    submission['Prediction'] += valid['pred_diff'].mean() - test_pred.mean()
    submission.to_csv(prefix / "submission_without_bias_grouped_corrected.csv", index=False)

In [ ]:
prepate_test_submsission(clf, prediction_diff_bias, prediction_diff_by_group, prefix='simple')

# HPO

In [ ]:
import optuna
import optuna.integration.lightgbm as lgb
optuna.__version__

In [ ]:
d_train = lgb.Dataset(X_train, label=y_train, feature_name=feature_names, categorical_feature=categorical_columns, free_raw_data=False)
d_valid = lgb.Dataset(X_valid, label=y_valid, feature_name=feature_names, categorical_feature=categorical_columns, free_raw_data=False)
gc.collect()

In [ ]:
boosting_static_params = {
        "objective": "regression",
        "metric": "l2",
        "verbosity": 0,
        # "boosting_type": "gbdt",
}

In [ ]:
study = optuna.create_study(direction="minimize")

In [ ]:
time_to_train = timedelta(hours=6).seconds

In [ ]:
model = lgb.train(
        params=boosting_static_params, 
        train_set=d_train, valid_sets=[d_valid], 
        early_stopping_rounds=100, num_boost_round=2000,
        verbose_eval=0, 
        study=study, time_budget=time_to_train,
        model_dir='/kaggle/working/boostings',
    )

In [ ]:
dump(study, "study.joblib")

In [ ]:
prediction_diff_bias, prediction_diff_by_group = calculate_valid_metrics(model, prefix='hpo')

In [ ]:
prepate_test_submsission(model, prediction_diff_bias, prediction_diff_by_group, prefix='hpo')

Retrain on full dataset

In [ ]:
print('Best parameters: ' + json.dumps(study.best_params, indent=4))

In [ ]:
import lightgbm as original_lightgbm

In [ ]:
model_retrained = original_lightgbm.train(
    params={**boosting_static_params, **best_params, 'learning_rate': 0.03}, 
    train_set=d_train, valid_sets=[d_valid],
    feature_name=feature_names, categorical_feature=categorical_columns,
    init_model=model,
    verbose_eval=400,
    early_stopping_rounds=600, num_boost_round=4000)

In [ ]:
prediction_diff_bias, prediction_diff_by_group = calculate_valid_metrics(model_retrained, prefix='hpo_retrained')

In [ ]:
prepate_test_submsission(model_retrained, prediction_diff_bias, prediction_diff_by_group, prefix='hpo_retrained')

# Feature importance

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
def plot_importance(model, importance_type='split'):
    """
    importance_type could be split or 'gain'
    """
    feat_importance = pd.DataFrame()
    feat_importance["feature"] = X_train.columns
    feat_importance[importance_type] = model.feature_importance(importance_type=importance_type)
    feat_importance = feat_importance.sort_values(by=importance_type, ascending=False).head(20)
    plt.figure(figsize=(20, 10))
    ax = sns.barplot(y="feature", x=importance_type, data=feat_importance)

In [ ]:
plot_importance(model_retrained, importance_type='split')

In [ ]:
plot_importance(model_retrained, importance_type='gain')